In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
import matplotlib.pyplot as plt

#%%
# Завантаження набору даних
with open('mod_05_topic_10_various_data.pkl', 'rb') as fl:
    datasets = pickle.load(fl)

# Перевірка доступних даних
print("Available datasets:", datasets.keys())

# Вибір набору даних Concrete
data = datasets['concrete']
print(data.head())

#%%
# Створення нової ознаки Components
data['Components'] = data.apply(lambda row: sum(row != 0), axis=1)

# Нормалізація даних
features = data.drop(columns=['Components', 'Cluster'], errors='ignore')

scaler = StandardScaler()
normalized_data = scaler.fit_transform(features)

#%%
# Визначення оптимальної кількості кластерів
kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(1, 10))
visualizer.fit(normalized_data)
visualizer.show()

# Отримання оптимальної кількості кластерів
optimal_k = visualizer.elbow_value_
print(f"Optimal number of clusters: {optimal_k}")

#%%
# Кластеризація методом k-середніх
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
labels = kmeans.fit_predict(normalized_data)

# Додавання міток до даних
data['Cluster'] = labels

#%%
# Розрахунок описової статистики кластерів
cluster_summary = data.groupby('Cluster').median()

# Розрахунок кількості об'єктів у кожному кластері
cluster_counts = data['Cluster'].value_counts()

#%%
# Додавання кількості об'єктів до звіту
cluster_summary['Count'] = cluster_counts

# Виведення звіту
print("Cluster Summary:")
print(cluster_summary)

# Додавання таблиці
table_data = cluster_summary.reset_index()

# Додавання таблиці до графіку
table = plt.table(cellText=table_data.values,
                  colLabels=table_data.columns,
                  cellLoc='center',
                  loc='bottom',
                  bbox=[0.0, -0.35, 1.0, 0.3])
plt.show()
#%%
# Після проведення кластеризації набору даних про бетон я отримала кілька важливих результатів, які можуть допомогти глибше зрозуміти структуру даних і характеристики бетону.

# 1. Оптимальна кількість кластерів:
# За результатами методу Elbow, оптимальна кількість кластерів для нашого набору даних становить 4. Це число вказує на найкраще розподілення даних на підгрупи, забезпечуючи гарний баланс між якістю кластеризації та складністю моделі.

# 2. Склад кластерів:
# Кластер 0: 89 об'єктів, середній цемент — 315.0, шлаковий матеріал — 20.00.
# Кластер 1: 83 об'єкти, середній цемент — 339.0, шлаковий матеріал — 0.00.
# Кластер 2: 98 об'єктів, середній цемент — 199.3, шлаковий матеріал — 182.45.
# Кластер 3: 157 об'єктів, середній цемент — 165.0, шлаковий матеріал — 97.10.

# 3. Кількість об'єктів у кластерах:
# Кластер 3 має найбільшу кількість об'єктів (157), а кластер 1 — найменшу (83).

# 4. Аналіз компонентів:
# Ознака Components, яка представляє кількість ненульових значень у кожному рядку, допомогла виявити, що кластер 3 має найвищу середню кількість компонентів (8.0), що може вказувати на різні підходи до виробництва або різні типи бетону в цьому кластері.

